# UNet: Convolutional Networks for Biomedical Image Segmentation

### Imports

In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds

### Download the Dataset

In [ ]:
dataset, info = tfds.load("oxford_iiit_pet:3.*.*", with_info=True)